# This is the script to extract Liver bounding!

In [7]:
import os
import nibabel as nib
len(os.listdir('../../Data/CT_Phase/Task_502_Phase_Data/'))
#len(os.listdir('../../Data/CT_Phase/Full_Image_Liver_07075/'))

263

In [29]:
import os
import nibabel as nib
import numpy as np
import skimage 
import matplotlib.pyplot as plt
from tqdm import tqdm
import SimpleITK as sitk
import nibabel as nib
import math
#print(os.listdir('../../Test_Data/Test_3D/'))
class ImageLoad:
    def __init__(self,input_path):
        """
        The path should only contain images
        """
        self.input_path = input_path


        #self.images_names = os.listdir('../../Test_Data/Test_3D/')
        self.images_names = os.listdir(self.input_path)
        self.images_num = len(self.images_names)
        self.image_path = [self.input_path + name for name in self.images_names]

    def image_load(self,image_path,reader='nib'):
        if reader == 'nib':
            return nib.load(image_path)
        elif reader == 'sitk':
            return sitk.ReadImage(image_path)
        else:
            raise ValueError("The reader should be either nib or sitk!")
        
    #def image_get_float(self):

  


class LiverBoundingBox:
    def __init__(self,liver_mask,liver_orig,out_path_box=None,file_name=None):
        """
        args:
            liver_mask: the liver mask|nib or sitk
            original_image: the original image|nib or sitk
        """
        self.liver_seg = liver_mask
        self.liver_orig = liver_orig
        self.out_path_box = out_path_box
        self.file_name = file_name
        # self.original_image = original_image
        # self.liver_mask_array = self._get_array(liver_seg)
        # self.original_arrary = self._get_array(original_image)
        # self.image_loader = image_loader
    def extract_liver(self,liver=True):
        """
            get either the liver or tumor region from the mask
            args:
                mask: mask of the liver and tumor
                liver: if True return the liver region, if False return the tumor region
        """
        mask = self._get_array(self.liver_seg)
        mask = mask.astype(int)
        print(mask.shape)
        
        if liver:
            mask[mask == 2] = 1
        else:
            mask[mask == 1] = 0
            mask[mask == 2] = 1
            mask = mask.astype(int)
            
        # only keep the largest connected component
        labeled = skimage.measure.label(mask, connectivity=2)

        labeled[labeled != 1] = 0
      
        mask = labeled
        print(mask.sum())

        return mask


        
    def get_liver_bounding_box(self,liver_mask):
        '''
    Function to generate bounding box for liver, from a binary liver mask
        args:
            liver_mask: binary mask of the liver

        returns:
            bbox: bounding box of the liver (min_row, min_col, min_slice, max_row, max_col, max_slice = bbox)
        '''

        # get the image_probs
        image_probs = skimage.measure.regionprops((liver_mask))

        # get the bounding box of the liver

        if len(image_probs) == 0:
            print(f'[WARNING] no liver found')
            self._recording_failing()
            return None

        ## find the adjacent box that contains the liver
        for props in image_probs:
            bbox = props.bbox
            min_row, min_col, min_slice, max_row, max_col, max_slice = bbox 
            print("this is range",min_row, min_col, min_slice, max_row, max_col, max_slice)
        return [min_row, min_col, min_slice, max_row, max_col, max_slice]

    def crop_scan(self,liver_bounding):
        """
        Crop the scan with the bounding box
        args:
            liver_bounding: the bounding box of the liver
        """
        liver_original = self._get_array(self.liver_orig)
        # get the bounding box of the liver
        min_row, min_col, min_slice, max_row, max_col, max_slice = list(map(self._check_range,liver_bounding))
        print('this is after check range',min_row, min_col, min_slice, max_row, max_col, max_slice)
        
        #crop the scan
        cropped_scan = liver_original[min_row:max_row,min_col:max_col,min_slice:max_slice]
        return cropped_scan
    
    def store_cropped_data(self,cropped_data):

        if not os.path.exists(self.out_path_box):
            os.mkdir(self.out_path_box)
            print('The path does not exist, create the path!')

        header = self.liver_orig.header
        affine = self.liver_orig.affine
        print("this is shape of cropped data",cropped_data.shape)
        cropped_image = nib.Nifti1Image(cropped_data, affine, header)


        
        nib.save(cropped_image, self.out_path_box + self.file_name)

    @staticmethod   
    def liver_detection(mask):
        if np.count_nonzero(mask) != 0:
            return True


    def _check_range(self,range_num):
        return max(0,range_num)
    
    def _get_array(self,image_file):
        return image_file.get_fdata()
    
    def _recording_failing(self):
        with open(self.out_path_box + 'failing_box.txt','a') as f:
            f.write(self.file_name + '\n')








NameError: name 'os' is not defined

In [12]:
image_lst = os.listdir('../../Data/CT_Phase/Full_Image_Liver_07075/')
image_liver_slice = {img:() for img in image_lst}

{'CILM_CT_101040_0000.nii.gz': (),
 'CILM_CT_102030_0000.nii.gz': (),
 'CILM_CT_104550_0000.nii.gz': (),
 'CILM_CT_104551_0000.nii.gz': (),
 'CILM_CT_104552_0000.nii.gz': (),
 'CILM_CT_107850_0000.nii.gz': (),
 'CILM_CT_107851_0000.nii.gz': (),
 'CILM_CT_111110_0000.nii.gz': (),
 'CILM_CT_111111_0000.nii.gz': (),
 'CILM_CT_111112_0000.nii.gz': (),
 'CILM_CT_111550_0000.nii.gz': (),
 'CILM_CT_111551_0000.nii.gz': (),
 'CILM_CT_111552_0000.nii.gz': (),
 'CILM_CT_111553_0000.nii.gz': (),
 'CILM_CT_114470_0000.nii.gz': (),
 'CILM_CT_114471_0000.nii.gz': (),
 'CILM_CT_114472_0000.nii.gz': (),
 'CILM_CT_116560_0000.nii.gz': (),
 'CILM_CT_116561_0000.nii.gz': (),
 'CILM_CT_121900_0000.nii.gz': (),
 'CILM_CT_121901_0000.nii.gz': (),
 'CILM_CT_125530_0000.nii.gz': (),
 'CILM_CT_125531_0000.nii.gz': (),
 'CILM_CT_125532_0000.nii.gz': (),
 'CILM_CT_129310_0000.nii.gz': (),
 'CILM_CT_129311_0000.nii.gz': (),
 'CILM_CT_129570_0000.nii.gz': (),
 'CILM_CT_134190_0000.nii.gz': (),
 'CILM_CT_134191_000

In [70]:
cropped_out_path = '../../Data/CT_Phase/Full_image_Liver_slice_07075'
#load the images
image_orig_load = ImageLoad("../../Data/CT_Phase/Full_Image_Liver/")
image_seg_load = ImageLoad("../../Data/CT_Phase/Task_502_Phase_Data/")
for i in range(image_orig_load.images_num):
    file_name = image_orig_load.images_names[i]
    image_orign,image_segg = image_orig_load.image_load(image_orig_load.image_path[i]),image_seg_load.image_load(image_seg_load.image_path[i])

    #finding bounding box
    liver_bbox = LiverBoundingBox(image_segg,image_orign,cropped_out_path,file_name)
    liver_mask = liver_bbox.extract_liver()
    if liver_bbox.liver_detection(liver_mask):
        liver_box_range = liver_bbox.get_liver_bounding_box(liver_mask)
        image_liver_slice[file_name] = liver_box_range
        
    else:
        liver_bbox._recording_failing()

(512, 512, 112)
834018
this is range 61 143 72 330 424 111
(512, 512, 56)
753106
this is range 43 146 19 348 416 54
(512, 512, 37)
204600
this is range 92 176 7 293 383 35
(512, 512, 61)
907715
this is range 94 169 2 416 387 58
(512, 512, 79)
598292
this is range 87 166 42 433 387 77
(512, 512, 59)
226024
this is range 103 178 0 387 368 12
(512, 512, 127)
470047
this is range 101 177 48 380 372 79
(512, 512, 48)
951642
this is range 46 140 5 387 419 43
(512, 512, 42)
789539
this is range 30 143 0 376 400 36
(512, 512, 48)
890230
this is range 40 176 5 387 414 43
(512, 512, 34)
492336
this is range 44 133 1 319 374 32
(512, 512, 60)
570101
this is range 27 114 0 341 384 15
(512, 512, 113)
2087630
this is range 43 127 6 332 376 108
(512, 512, 125)
657710
this is range 43 125 49 328 374 80
(512, 512, 142)
783531
this is range 60 144 85 325 386 140
(512, 512, 65)
741745
this is range 62 142 8 321 384 62
(512, 512, 142)
776687
this is range 59 140 86 322 383 141
(512, 512, 38)
447880
this i

In [61]:
def save_liver_slice(image_path,image_name,cropped_out_path,slice_margin=0):
    """
    args:
        image_path: the path of the image
        image_name: the name of the image
        slice_margin: the margin of the slice
    """
    image = nib.load(image_path)
    image_data = image.get_fdata()
    image_shape = image_data.shape
    print(image_shape,666)
    min_row, min_col, min_slice, max_row, max_col, max_slice = image_liver_slice[image_name]
    print('min and max slice',min_slice,max_slice)
    min_slice = max(0,math.floor(min_slice - min_slice*slice_margin))
    print('fucking max slice',max_slice)
    max_slice = min(image_shape[2],math.ceil(max_slice + max_slice* slice_margin))
    print("fucking after",max_slice)
    print(min_slice,max_slice)
    image_data = image_data[:,:,min_slice:max_slice]
    print(image_data.shape)
    image = nib.Nifti1Image(image_data, image.affine, image.header)
    nib.save(image, cropped_out_path + '/' + image_name)
    print('saved!')

In [69]:
image_orign = nib.load('../../Data/CT_Phase/Full_Image_Liver_07075/CILM_CT_101040_0000.nii.gz')
image_orign.shape

(521, 521, 112)

In [71]:
import matplotlib.pyplot as plt
image_path = '../../Data/CT_Phase/Full_Image_Liver_07075/CILM_CT_101040_0000.nii.gz'
image_name = 'CILM_CT_101040_0000.nii.gz'
cropped_out_path = '../../Data/CT_Phase/Full_image_Liver_slice_07075'

os.makedirs(cropped_out_path,exist_ok=True)
save_liver_slice(image_path,image_name,cropped_out_path,slice_margin=0.2)


#save all slice that includes liver
for i in tqdm(range(image_orig_load.images_num)):
    file_name = image_orig_load.images_names[i]
    image_path = image_orig_load.image_path[i]
    save_liver_slice(image_path,file_name,cropped_out_path,slice_margin=0.2)



(521, 521, 112) 666
min and max slice 72 111
fucking max slice 111
fucking after 112
57 112
(521, 521, 55)
saved!


  0%|          | 0/263 [00:00<?, ?it/s]

(512, 512, 112) 666
min and max slice 72 111
fucking max slice 111
fucking after 112
57 112
(512, 512, 55)


  0%|          | 1/263 [00:00<03:41,  1.18it/s]

saved!
(512, 512, 56) 666
min and max slice 19 54
fucking max slice 54
fucking after 56
15 56
(512, 512, 41)


...

saved!
(512, 512, 67) 666
min and max slice 4 51
fucking max slice 51
fucking after 62
3 62
(512, 512, 59)


100%|█████████▉| 262/263 [04:20<00:01,  1.08s/it]

saved!
(512, 512, 65) 666
min and max slice 10 55
fucking max slice 55
fucking after 65
8 65
(512, 512, 57)


100%|██████████| 263/263 [04:21<00:00,  1.01it/s]

saved!


In [48]:
img = nib.load('../../Data/CT_Phase/Full_image_Liver_slice_07075/CILM_CT_246501_0000.nii.gz').get_fdata()
#plt.imshow(img[:,:,0],cmap='gray')

min and max slice 345 544
276 93
(539, 539, 0)
saved!


In [19]:
a.shape

(258, 288, 30)